In [ ]:

#BEGIN
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', **{ "size":16}) #**{,,'serif':['Palatino']
rc('text', usetex=True)
import numpy.random as rnd
rnd.seed()
import scipy.sparse as sp
import scipy.sparse.linalg as sla
import scipy.linalg as la
from scipy.optimize import brentq as solve
from scipy.integrate import odeint as ode
from scipy.special import gamma
np.set_printoptions(precision = 3, suppress = 1)
from matplotlib import cm
color = cm.get_cmap("Blues")

def get_Ising_terms():
    Xs = sum([sp.kron(np.identity(2 ** (L-j-1)), sp.kron(X, np.identity(2 ** j)) ) for j in range(L)])
    ZZ = sp.kron(Z, Z)
    ZZs = sum([sp.kron(np.identity(2 ** (L-j-2)), sp.kron(ZZ, np.identity(2 ** j)) ) for j in range(L-1)])
    return Xs, ZZs

# Fermion mapping of TFIM

The 2D Ising model is in the same universality class of the transverse field Ising model
#### $$ H =  - \sum_{j=1}^L h X_j -   \sum_{j=1}^{L-1}J Z_j Z_{j+1}$$

This quantum spin chain also offers (arguably) the simplest route to the exact solution, via a free fermion mapping. 

Unless otherwise specified, we will consider the model with open boundary condition, on a strip of length $L$. 

## The Jordan Wigner transform

- Show that the operators 
#### $$\gamma_j = Z_j \prod_{k < j} X_k \,,\, \eta_j = Y_j \prod_{k < j} X_k $$
satisfy the anticommutation relations of Majorana fermions 
#### $$ \{ \gamma_j, \gamma_k \} = \{ \eta_j, \eta_k \}  =2 I \delta_{jk} $$
$\gamma$'s and $\eta$'s anti-commute.

- (For those who never saw Majorana's) Verify that 
#### $$c_j = \frac12 (\gamma_j + \mathrm{i}\eta_j) , c^\dagger = \frac12 (\gamma_j - \mathrm{i}\eta_j) $$
satisfy the anticommutation relations of complex fermion creation and annihilation operators. 
- Show that 

#### $$ \mathrm{i} \gamma_j \eta_j  = X_j  \,,\,  \mathrm{i} \eta_j \gamma_{j+1} = Z_j Z_{j+1} $$

If we place $\eta_j$ between $\gamma_j$ and $\gamma_{j+1}$, the TFIM is equivalent to a Majorana fermion chain with nearest neighbor coupling and staggered coupling constants (see illustration below). It is also known as the Kitaev chain.


In [ ]:
L = 4
plt.figure(1, figsize=(8,1.2))
for j in range(1,L):
    #plt.plot([j+.5, j+.5],[-1,1],"k")
    plt.plot([j+.25, j+.75],[0,0], lw=3, c = "r")
    plt.text(j+.5, -.2, r"$J$", va = "top", ha = "center" )
for j in range(1,L+1):
    plt.plot([j-.25, j+.25],[0,0], lw=3, c = "k")
    plt.text(j-.25, .2, r"$\gamma_{%d}$" % j, va = "bottom", ha = "center" )
    plt.text(j+.25, .2, r"$\eta_{%d}$" % j, va = "bottom", ha = "center" )
    #plt.text(j-.25, .6, r"$\psi_{%d}$" % (2 * j-1), va = "bottom", ha = "center" )
    #plt.text(j+.25, .6, r"$\psi_{%d}$" % (j*2), va = "bottom", ha = "center" )
    
    plt.text(j, -.2, r"$h$", va = "top", ha = "center" )
plt.plot(np.arange(1,L+1) - .25, np.zeros(L), "o", c = "b", ms = 10)
plt.plot(np.arange(1,L+1) + .25, np.zeros(L), "o", c = "b", ms = 10)

plt.yticks([])
plt.xticks(np.arange(L)+1)
plt.ylim(-1,1)

###  Egde zero modes

We will take a break from the exact solutin, and study the Majorana chain in the two limits, $h \gg J$ (trivial paramagnet) and $ J \gg h$ (trivial ferromagnet).
- When $ J = 0$, the Hamiltonian is a sum of commuting Majorana "pairings": 
#### $$H \propto \mathrm{i} \gamma_1\eta_1 + \mathrm{i} \gamma_2 \eta_2 + \dots$$
Can you find its ground state (you can use the complex fermions if want, but try not to go back to the spin operators)
- When $h = 0$, how does the pairing change? Yes, everyone is paired, except the first one and the last one. They are known as edge *zero modes*. **Zero modes are operators that commute with the Hamiltonian.** How many ground states are there? 
- The degeneracy can be lifted by closing the chain, i.e., adding a $\propto \mathrm{i} \psi_{2L} \psi_1$ term. Conclude that the periodic Kitaev chain is **not** the periodic TFIM.

## Solving Majorana free fermion models (lazy way)

We now diagonalize the Kitaev chain using general methods for Majorana free fermion models, i.e., throwing the Hamiltonian, as a fermion bilinear form, to the computer. The task of the computer is then to diagonalize a $2L\times2L$ matrix, which is polynomially hard. In comparison, the many-body Hamiltonian itself is a $2^L \times 2^L$ matrix.

We adopte a more generic notation: 
#### $$ \psi_1, \psi_2, \psi_3, \psi_4,  \dots, \psi_{2L-1}, \psi_{2L} = \gamma_1, \eta_1, \gamma_2, \eta_2, \dots , \gamma_L, \eta_L $$

- The TFIM Hamiltonian can be written as 
#### $$ H = \frac{\mathrm{i}}2 \sum \psi_j A_{jk} \psi_k $$
where $A_{jk}$ is a $2L\times 2L$ real antisymmetric matrix. Can you describe it explicitly (answer hidden in the code below)? 

- Now consider a general $A \in so(2n)$ like above. A standard result in linear algebra is that, there is an orthogonal matrix $U$ that transforms $A$ into the following normal form:
#### $$ U^T A U =  \begin{pmatrix} & D  \\  -D & \end{pmatrix} \,,\, 
D = \mathrm{diag}(\epsilon_1, \dots, \epsilon_L)  $$ 
where $E $ and $\epsilon_j$'s are positive real numbers (the code below is equivalent to a constructive proof). **In fact**, $\pm \mathrm{i} \epsilon_\ell, \ell = 1, \dots, L$ are the eigenvalues of $A$, and the corresponding eigenvectors are the $\ell$-th and $(\ell + L)$-th columns of $U$, respectively.

Using this result, show that 
#### $$ H = \mathrm{i} \sum_{\ell=1}^L \epsilon_k \tilde{\psi}_{\ell} \tilde{\psi}_{\ell+L}  \,,\, \tilde{\psi}_\ell = \sum_{j} U_{j\ell} \psi_j \,. $$

**Important** verify that $\tilde{\psi}_\ell$'s satisfy the same Majorana anticommutation relations, because $U$ is orthogonal. This is why we took the trouble of not diagonalizing $A$ as one would usually done. (See below for more discussion on this)

- What can you say about the spectrum of $H$ now? (see answer below)

In [ ]:
def getA():
    """ construct the hopping matrix for a Kitaev chain """
    A = np.zeros((L * 2, L * 2), dtype=float)
    A[np.arange(L) * 2, np.arange(L) * 2+1] = h
    A[np.arange(L) * 2 + 1, np.arange(L) * 2] = -h
    A[np.arange(L-1) * 2 + 1, np.arange(L-1) * 2+2] = J
    A[np.arange(L-1) * 2 + 2, np.arange(L-1) * 2+1] = -J
    return A
def standardform(A):
    """ compute the standard form of a real antisymmetric matrix A """
    u, epsilons ,v = la.svd(A)
    U = np.hstack([u[:,::2], v[::2].T])
    
    return epsilons[::2], U

X = np.array([0,1,1,0]).reshape(2,2)
Z = np.array([1,0,0,-1]).reshape(2,2)
iY =  np.array([0,1,-1,0]).reshape(2,2)
def get_Ising_terms():
    Xs = sum([sp.kron(np.identity(2 ** (L-j-1)), sp.kron(X, np.identity(2 ** j)) ) for j in range(L)])
    ZZ = sp.kron(Z, Z)
    ZZs = sum([sp.kron(np.identity(2 ** (L-j-2)), sp.kron(ZZ, np.identity(2 ** j)) ) for j in range(L-1)])
    return Xs, ZZs

### Benchmark with ED

*This is what you should always do before you use a clever method in your research*

In [ ]:
L = 6
J = 1
h = 2.
# JW
# ------------------
A = getA()
epsilons, U = standardform(A)
#--------------

#ED
#------------------
Xs, ZZs = get_Ising_terms()
H = Xs * h + ZZs * J

w, v = sla.eigsh(H, which = "SA", k=2)
#----------------
print("ground state by ED", w[0])
print("ground state by JW", -np.sum(epsilons))
print("gap by ED", w[1]-w[0])
print("gap by JW", 2 * epsilons[-1])

A look at the A matrix 

In [ ]:
plt.pcolor(A, cmap = "RdBu")
plt.ylim(2 *L, 0)
plt.colorbar()

Check that the standard form works

In [ ]:
N = U.T @ A @ U
plt.pcolor(N, cmap="RdBu")
plt.ylim(L*2,0)
plt.colorbar()
print(epsilons)
print(N[np.arange(L), np.arange(L) + L])

## Elementary excitations

We have seen that the many-body eigenstates of $H$ are of form 
#### $$ \pm \epsilon_1 \pm \epsilon_2 \dots  \pm \epsilon_L \,. $$
$2\epsilon_j$'s are the energy of elementary excitations in this system. We want to understand the low-lying excitations in a large system ($L\gg 1$). 

Before further analytical considerations, let us look at them numerically.

For this let us parametrize 
#### $$J = \sin(\theta), h = \cos(\theta)$$
so that $\theta \to \pi / 2 - \theta$ is the duality transform and $\theta= \pi / 4$ is critical.
We sweep the phase diagram and compute the excitation energies.

Plotting those against $\theta$, we obtain one of the most important plots of this TD. Take some time to observe it and make many comments (some questions are below).

In [ ]:
L = 100
#h = 1
#Js = np.linspace(0,2, 50)
ths = np.linspace(0, np.pi / 2, 101)
Eps = np.zeros((len(ths), 100), dtype=float)
for k,th in enumerate(ths):
    J, h = np.sin(th), np.cos(th)
    A = getA()
    epsilons, U  = standardform(A)
    Eps[k,:] = epsilons[::-1]

for j in range(min(20, L)):
    l = "$k = %d$" % (j+1) if j in [0, 19] else None
    plt.plot(ths, Eps[:, j], c = color(1-j / 30), label=l)
plt.xlabel(r"$\theta = \arctan(J/h)$")
plt.ylabel(r"$\epsilon_k$")
plt.legend()
plt.xticks([0, np.pi/4, np.pi/2], ["0", r"$\pi/4$", "$\pi/2$"])

- What distinguishes the two phases and the critical point?
- Would you call the ferro phase gapped or gapless?
- Why is the lowest excitation so close to $0$? Do you remember how close (from ED)? 
- Can you infer the behavior of the correlation length near criticality?

## The tight-binding model

### Infinite chain

- Using Bloch theorem, diagonalize the hopping matrix $A$ in an **infinite** chain. Conclude that the elementary excitation associated with $q \in [0, \pi)$ is
$$  \epsilon_q =  |h - J e^{\mathrm{i} q}|   $$

Let us check that against numerics in an open chain of size $L$. 
- Which $q$'s would we choose (naively)?
- How does that work?

In [ ]:
L = 32
fig, ax = plt.subplots(1,3, figsize = (8,3), sharex = "all", sharey = "all")
text = ["$h > J$", "$h = J$", "$h < J$"]
for i, th in enumerate([np.pi/6, np.pi/4, np.pi/3]):
    h, J = np.cos(th), np.sin(th)
    A = getA()
    epsilons, U  = standardform(A)
    epsilons = epsilons[::-1]
    qs = np.arange(L) / L * np.pi 
    cut = 32
    plt.sca(ax[i])
    ls = np.arange(L) + 1
    plt.plot(ls, epsilons, ".", c = "b")
    plt.plot(ls, np.abs(h - J * np.exp(1.j * qs)), c = "k")
    plt.xlabel(r"$\ell$",labelpad=-1)
    plt.xticks([1, 32])
    plt.text(1, 1.3, text[i])
    if not i: plt.ylabel(r"$\epsilon_{\ell}$")
plt.subplots_adjust(wspace=.05)

Bloch theory fails miserably with the lowest mode in the $h < J$ phase. So let us look at the single particle wave function of that mode. What do you see? Is it something we encountered before? 

Yes these are the edge zero modes. When $h \ll J$, they are completely localized at $\gamma_1$ and $\eta_L$. When $h < J$, they are exponentially localized near the left and right edge. 

- Can we predict the exponential decay rate in the $L\to\infty$ limit, by writing a trial wave function (hint: numerics below)
- Do we understand why zero modes only exists when $J > h$?
- Can we understand the exponential degeneracy?
- As mentioned above, zero modes are operators that commute with the Hamiltonian. What are the operators we are taking about here? Argue that they **almost** commute with the Hamiltonian, up to an error that is exponentially small in system size.

In [ ]:
L = 32
th = np.pi / 3
h, J = np.cos(th), np.sin(th)
A = getA()
epsilons, U  = standardform(A)
xs = np.arange(L) + 1
f, ax= plt.subplots(1,2,figsize = (6,2.5), sharex = "all", sharey = "all")
cs =["r", "b"]
for i in range(2):
    plt.sca(ax[i])
    plt.plot(xs,U[::2,-1 + L * i], c = cs[i],label=r"$\gamma$")
    plt.plot(xs,U[1::2,-1 + L * i], "--", c = cs[i], label=r"$\eta$")
    plt.xticks([1, 32])
    plt.xlabel("$j$",labelpad=-1)
    plt.legend()
    if i ==0: plt.ylabel("amplitude")


## Final thoughts

- What is the spin operator $Z_j$ in terms of Majorana's? 

This is the price of the JW transform. It transforms the order parameter into a rather complex non-local operator, (sometimes known as a string operator). Meanwhile, the Ising transition acquires a *topological* nature: the ferromagnetic phase is no longer characterized by a local order parameter, but the existence of edge zero modes, which are a general signature of nontrivial topological phases; more precisely, *symmetry protected* topological (SPT) phases. 

## (For the curious) Why does J-W transform work? 
- Take a term in the TFIM, say $X_1$, and compute its commutator with other terms of the TFIM. Then take the resulting operators and compute their commutator with other terms of the TFIM, so on and so forth.
- Do you see a pattern? All the operators generated are of type 
#### $$ O_j X_{j+1} X_{j+2} \dots X_{k-1} O'_k $$
where $O, O'$ are either $Y$ or $Z$. 

- How many are they in a chain of size $L$? It is very few compared to the dimension of the space of all operators ($4^L$).

A generic model will usually generate exponentially many operators, if not all of them. When much fewer are generated, the model is rather easily solveable (the inverse is not true; e.g., Bethe-Ansatz integrable models).  

Finally, we note that the global purpose of this course is *not* to solve models exactly (it is quite the opposite). 